In [1]:
import duckdb
import pandas as pd
import os
import json
import sqlite3 as lite
import hashlib
from sklearn.model_selection import train_test_split

# DiverseVul

In [2]:
df_diversevul = duckdb.query('''SELECT * FROM read_json('../../datasets/diversevul_dataset.json', auto_detect=True)''').to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
file_path = "../../datasets/diversevul_20230702_metadata.json"
data = []
a = 0
with open(file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line:  # Skip empty lines
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError as e:
                a=a+1
                print(f"Skipping line due to JSONDecodeError: {e}")

metadata_dv = pd.DataFrame(data)

def extract_cwe_number(cwe_list):
    if len(cwe_list) > 0:
        return int(cwe_list[0].split('-')[1])
    return None

In [4]:
df_dv = df_diversevul.merge(metadata_dv[['commit_id', 'CVE', 'commit_url']], on='commit_id', how='left')
df_dv = df_dv.drop(columns=['commit_id','hash','size','message','commit_url','project'])
df_dv['year'] = df_dv['CVE'].str.extract(r'CVE-(\d{4})')
df_dv['year'] = df_dv['year'].fillna(0)
df_dv['year'] = df_dv['year'].astype(int)
df_dv = df_dv.drop(columns=['CVE'])
df_dv['cwe_number'] = df_dv['cwe'].apply(lambda x: extract_cwe_number(x))
df_dv['cwe_number'] = df_dv['cwe_number'].fillna(0).astype(int)
df_dv = df_dv.drop(columns=['cwe'])
df_dv = df_dv.rename(columns={'cwe_number': 'cwe','func':'text','target':'label'})
df_dv['source'] = "diversevul"

# CrossVul

In [5]:
root_dir = "../../datasets/dataset_final_sorted"
metadata_path = os.path.join(root_dir, "metadata.json")

with open(metadata_path, "r") as f:
    metadata = json.load(f)

database_name_to_year = {}
for entry in metadata:
    year = entry["cve"].split("-")[1]
    for file_info in entry["files"]:
        database_name_to_year[file_info["database_name"]] = year

file_paths = []
labels = []
file_contents = []
cwe_numbers = []
years = []

for root, dirs, files in os.walk(root_dir):
    for file in files:
        if "c" in root and (file.startswith("bad_") or file.startswith("good_")):
            file_path = os.path.join(root, file)
            label = 1 if file.startswith("bad_") else 0
            
            with open(file_path, "r") as f:
                content = f.read()
            
            cwe_number = os.path.basename(os.path.dirname(os.path.dirname(file_path)))
            if cwe_number.startswith("CWE-"):
                cwe_number = cwe_number[4:]
            else:
                cwe_number = "None"
            
            database_name = os.path.splitext(file)[0]  # Get the database name without extension
            year = database_name_to_year.get(database_name, "Unknown")
            
            file_paths.append(file_path)
            labels.append(label)
            file_contents.append(content)
            cwe_numbers.append(cwe_number)
            years.append(year)

df_cv = pd.DataFrame({
    "label": labels,
    "text": file_contents,
    "cwe": cwe_numbers,
    "year": years,
    "source": "crossvul"
})

# CVEFixes

In [6]:
df_cvefixes_old = pd.read_json("../data/cvefixes_old.json")
df_cvefixes_22 = pd.read_json("../../datasets/cvefixes_data_22.json")
df_cvefixes_22['year'] = 2022
df_cvefixes_23 = pd.read_json("../../datasets/cvefixes_data_23.json")
df_cvefixes_23['year'] = 2023
df_cvefixes_24 = pd.read_json("../../datasets/cvefixes_data_24.json")
df_cvefixes_24['year'] = 2024
df_cf = pd.concat([df_cvefixes_old, df_cvefixes_22, df_cvefixes_23, df_cvefixes_24], ignore_index=True)
df_cf['target'] = (df_cf['before_change'] == 'True').astype(int)
df_cf = df_cf.drop(columns=['name','signature','nloc','parameters','token_count','programming_language','repo_name','published_date','before_change'])
df_cf['cwe_id'] = df_cf['cwe_id'].str.extract(r'CWE-(\d+)')
df_cf['cwe_id'] = pd.to_numeric(df_cf['cwe_id'], errors='coerce')
df_cf = df_cf.dropna(subset=['cwe_id'])
df_cf['cwe_id'] = df_cf['cwe_id'].astype(int)
df_cf = df_cf.reset_index(drop=True)
df_cf = df_cf.rename(columns={'cwe_id': 'cwe','code':'text','target':'label'})
df_cf['source'] = "cvefixes"

# Merge

In [51]:
df = pd.concat([df_dv, df_cv, df_cf], ignore_index=True)

# Remove duplicates and take < 4000

In [52]:
df['text_clean'] = df['text'].str.replace(r'[\s\n\t\r]', '', regex=True)

# Compute MD5 hash of the normalized text
df['md5_hash'] = df['text_clean'].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())

# De-duplicate based on MD5 hash
nondup_df = df.drop_duplicates(subset=['md5_hash'], ignore_index=True)

# Drop temporary columns
nondup_df = nondup_df.drop(['text_clean'], axis=1)
nondup_df = nondup_df.rename(columns={"md5_hash":"hash"})
df = df.drop(['text_clean', 'md5_hash'], axis=1)

# Output results
print("Original merged df length: ", len(df))
print("Non duplicated df length:", len(nondup_df))
print("Duplicate entries: ", len(df) - len(nondup_df))

Original merged df length:  362758
Non duplicated df length: 340502
Duplicate entries:  22256


In [53]:
print("non duplicated df length (all):", len(nondup_df))
lengths = nondup_df['text'].apply(len)
nondup_df = nondup_df[lengths < 4000]
print("filtered non-duplicated df length (< 4000):", len(nondup_df))

non duplicated df length (all): 340502
filtered non-duplicated df length (< 4000): 312542


# Sort by year

In [54]:
nondup_df['year'] = pd.to_numeric(nondup_df['year'], errors='coerce')
nondup_df['year'] = nondup_df['year'].fillna(0).astype(int)
nondup_df.loc[nondup_df['label'] == 0, 'cwe'] = 0
nondup_df = nondup_df.sort_values(by='year')
nondup_df['cwe'].replace('None', 0, inplace=True)

In [55]:
df_1 = nondup_df[nondup_df['label'] == 1]
df_0 = nondup_df[nondup_df['label'] == 0]

# Take top 5 CWE

In [56]:
cwe_counts = df_1['cwe'].value_counts()

In [57]:
def take_random_entries(df, n):
    sampled_df = df_0.sample(n, random_state=42)
    df_remaining = df.drop(sampled_df.index)
    return sampled_df, df_remaining

In [58]:
df_1_125 = nondup_df[nondup_df['cwe'] == 125]
df_1_787 = nondup_df[nondup_df['cwe'] == 787]
df_1_119 = nondup_df[nondup_df['cwe'] == 119]
df_1_20 = nondup_df[nondup_df['cwe'] == 20]
df_1_416 = nondup_df[nondup_df['cwe'] == 416]
df_1 = df_1[~df_1['cwe'].isin([125,787,119,20,416])]

In [59]:
df_0_125, df_0 = take_random_entries(df_0, len(df_1_125))
df_0_787, df_0 = take_random_entries(df_0, len(df_1_787))
df_0_119, df_0 = take_random_entries(df_0, len(df_1_119))
df_0_20, df_0 = take_random_entries(df_0, len(df_1_20))
df_0_416, df_0 = take_random_entries(df_0, len(df_1_416))

In [60]:
train_1_125, test_1_125 = train_test_split(df_1_125, test_size=0.1, random_state=42)
train_0_125, test_0_125 = train_test_split(df_0_125, test_size=0.1, random_state=42)
train_125 = pd.concat([train_1_125, train_0_125], ignore_index=True)
test_125 = pd.concat([test_1_125, test_0_125], ignore_index=True)
df_0 = pd.concat([df_0, train_0_125], ignore_index=True)
df_1 = pd.concat([df_1, train_1_125], ignore_index=True)
train_125.to_json("train_125.json")
test_125.to_json("test_125.json")

In [61]:
train_1_787, test_1_787 = train_test_split(df_1_787, test_size=0.1, random_state=42)
train_0_787, test_0_787 = train_test_split(df_0_787, test_size=0.1, random_state=42)
train_787 = pd.concat([train_1_787, train_0_787], ignore_index=True)
test_787 = pd.concat([test_1_787, test_0_787], ignore_index=True)
df_0 = pd.concat([df_0, train_0_787], ignore_index=True)
df_1 = pd.concat([df_1, train_1_787], ignore_index=True)
train_787.to_json("train_787.json")
test_787.to_json("test_787.json")

In [62]:
train_1_119, test_1_119 = train_test_split(df_1_119, test_size=0.1, random_state=42)
train_0_119, test_0_119 = train_test_split(df_0_119, test_size=0.1, random_state=42)
train_119 = pd.concat([train_1_119, train_0_119], ignore_index=True)
test_119 = pd.concat([test_1_119, test_0_119], ignore_index=True)
df_0 = pd.concat([df_0, train_0_119], ignore_index=True)
df_1 = pd.concat([df_1, train_1_119], ignore_index=True)
train_119.to_json("train_119.json")
test_119.to_json("test_119.json")

In [63]:
train_1_20, test_1_20 = train_test_split(df_1_20, test_size=0.1, random_state=42)
train_0_20, test_0_20 = train_test_split(df_0_20, test_size=0.1, random_state=42)
train_20 = pd.concat([train_1_20, train_0_20], ignore_index=True)
test_20 = pd.concat([test_1_20, test_0_20], ignore_index=True)
df_0 = pd.concat([df_0, train_0_20], ignore_index=True)
df_1 = pd.concat([df_1, train_1_20], ignore_index=True)
train_20.to_json("train_20.json")
test_20.to_json("test_20.json")

In [64]:
train_1_416, test_1_416 = train_test_split(df_1_416, test_size=0.1, random_state=42)
train_0_416, test_0_416 = train_test_split(df_0_416, test_size=0.1, random_state=42)
train_416 = pd.concat([train_1_416, train_0_416], ignore_index=True)
test_416 = pd.concat([test_1_416, test_0_416], ignore_index=True)
df_0 = pd.concat([df_0, train_0_416], ignore_index=True)
df_1 = pd.concat([df_1, train_1_416], ignore_index=True)
train_416.to_json("train_416.json")
test_416.to_json("test_416.json")

In [65]:
df = pd.concat([df_0, df_1], ignore_index=True)
df.to_json("train_all.json")